In [1]:
import Dataset
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append("/home/user/elwakeely1/Cuda_Vision_Lab-/mast3r")

/home/nfs/inf6/data/datasets/ThermalDBs/Freiburg

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Initialize dataset
train_dataset = Dataset.Freiburg_dataset(root_dir="/home/nfs/inf6/data/datasets/ThermalDBs/Freiburg",split="train")
print(len(train_dataset))


# # Get images from dataset
# img_ir_aligned, img_rgb, img_rgb_labels = train_dataset.__getitem__(6)

# # Create a figure with 3 subplots
# fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# # Display each image
# axes[0].imshow(img_ir_aligned, cmap="gray")  # Thermal Image
# axes[0].set_title("IR Aligned")
# axes[0].axis("off")

# axes[1].imshow(img_rgb)  # RGB Image
# axes[1].set_title("RGB")
# axes[1].axis("off")

# axes[2].imshow(img_rgb_labels)  # Labels Image
# axes[2].set_title("RGB Labels")
# axes[2].axis("off")

# plt.show()


20854


In [4]:
import torch
print(torch.version.cuda)


11.8


In [5]:
import os
import glob
import torch
import numpy as np
import matplotlib.pyplot as pl
from mast3r.model import AsymmetricMASt3R
from mast3r.fast_nn import fast_reciprocal_NNs
from dust3r.inference import inference
from dust3r.utils.image import load_images



Warning, cannot find cuda-compiled version of RoPE2D, using a slow pytorch version instead


In [ ]:
device = 'cuda'
model_name = "naver/MASt3R_ViTLarge_BaseDecoder_512_catmlpdpt_metric"

# Load pre-trained MASt3R model
model = AsymmetricMASt3R.from_pretrained(model_name).to(device)

# Process consecutive frames
for i_1 in range(len(train_dataset) - 1):  # Process (I_t, I_t+1)
    img1_path, img2_path = str(train_dataset[i_1]), str(train_dataset[i_1 + 1])

    # Load and preprocess images
    images = load_images([img1_path, img2_path], size=512)
    output = inference([tuple(images)], model, device, batch_size=1, verbose=False)

    # Get predictions (depth, pose, intrinsics, descriptors)
    view1, pred1 = output['view1'], output['pred1']
    view2, pred2 = output['view2'], output['pred2']
    print("pred1 keys:", pred2.keys())
    # print("view1 keys:", view1.keys())
    # # depth is third Z-coordinates of pts3d -> represents the 3d coordinates  
    pts3d1 = pred1['pts3d'].squeeze(0).detach()
    pts3d2 = pred2['pts3d_in_other_view'].squeeze(0).detach()
    # pts3d1,pts3d2 = pred1['pts3d'].squeeze(0).detach() ,pred2['pts3d'].squeeze(0).detach()
    
    desc1, desc2 = pred1['desc'].squeeze(0).detach(), pred2['desc'].squeeze(0).detach()
    
    # find 2D-2D matches between the two images
    matches_im0, matches_im1 = fast_reciprocal_NNs(desc1, desc2, subsample_or_initxy1=8,
                                            device=device, dist='dot', block_size=2**13)

    # ignore small border around the edge
    H0, W0 = view1['true_shape'][0]
    valid_matches_im0 = (matches_im0[:, 0] >= 3) & (matches_im0[:, 0] < int(W0) - 3) & (
        matches_im0[:, 1] >= 3) & (matches_im0[:, 1] < int(H0) - 3)

    H1, W1 = view2['true_shape'][0]
    valid_matches_im1 = (matches_im1[:, 0] >= 3) & (matches_im1[:, 0] < int(W1) - 3) & (
        matches_im1[:, 1] >= 3) & (matches_im1[:, 1] < int(H1) - 3)

    valid_matches = valid_matches_im0 & valid_matches_im1
    matches_im0, matches_im1 = matches_im0[valid_matches], matches_im1[valid_matches]
    #  Depth of the first image is the z coordinate of the pointmap
    # image tensors (and depth maps) follow (height, width, channels)
    # Extract pixel coordinates (i, j)
    i_1 = torch.tensor(matches_im0[:, 1])  # x-coordinates (width)
    j_1 = torch.tensor(matches_im0[:, 0])  # y-coordinates (height)

    i_2 = torch.tensor(matches_im1[:, 1])  # x-coordinates (width)
    j_2 = torch.tensor(matches_im1[:, 0])  # y-coordinates (height)

    X_n = pts3d1[i_1, j_1, :]
    X_m = pts3d2[i_2,j_2,:]
    
    # Depth
    D =pts3d1[i_1, j_1, 2] 
    
    # Intrinsics 
    # Compute pixel-coordinate matrix
    pixel_coords = torch.stack([i_1 * D, j_1 * D, D], dim=1)  
    
    # Solve for K using least squares
    K, _, _, _ = torch.linalg.lstsq(X_n, pixel_coords)
    
    # Pose
    # Convert X^n to homogeneous coordinates
    ones = torch.ones(X_n.shape[0], 1)
    X_n_h = torch.cat([X_n, ones], dim=1)  # Shape: (N, 4)
    
    # Solve for P_n using least squares
    P_n, _, _, _ = torch.linalg.lstsq(X_m, X_n_h)  # (3, 4)
    P_n = P_n.T
    print(P_n.shape)

    
    

<PIL.PngImagePlugin.PngImageFile image mode=I size=1920x650 at 0x7F85B31AB5B0>
<PIL.PngImagePlugin.PngImageFile image mode=I size=1920x650 at 0x7F8563D89A60>
>> Loading a list of 2 images
 - adding /home/nfs/inf6/data/datasets/ThermalDBs/Freiburg/train/seq_00_day/00/fl_rgb/fl_rgb_1570722156_952177040.png with resolution 1920x650 --> 512x160
 - adding /home/nfs/inf6/data/datasets/ThermalDBs/Freiburg/train/seq_00_day/00/fl_rgb/fl_rgb_1570722157_2840648400.png with resolution 1920x650 --> 512x160
 (Found 2 images)
pred1 keys: dict_keys(['conf', 'desc', 'desc_conf', 'pts3d_in_other_view'])
torch.Size([4, 3])
